In [1]:
import numpy as np
import pandas as pd
import math
import scipy
import sys
from tqdm import tqdm
import functools

import portion as P

interval = P.empty()
for i, j in [(0, 1), (2, 4), (6, 7), (3, 5)]:
    interval = interval | P.closed(i, j)
print(interval)

[0,1] | [2,5] | [6,7]


In [55]:
##print a map!!
def pprint(map):
    for y in range(map.shape[0]):
        l = ''
        for x in range(map.shape[1]):
            if map[y,x] == 0:
                l += '.'
            elif map[y,x] == 5:
                l += 'S'
            elif map[y,x] == 1:
                l += 'B'
            elif map[y,x] == 3:
                l += '#'
        print(l)

def generate_pts_with_distance_in_row(pt,distance,map,row):
    d = distance - abs(pt[0]-row)
    return pt[1]-d,pt[1]+d

def generate_pts_with_distance(pt,distance,map):
    ret = []
    for i in range(-distance,distance+1):
        for j in range(-distance,distance+1):
            if not (i == 0 and j == 0) and (abs(i)+abs(j) <= distance):
                try:
                    if map[pt[0]+i, pt[1]+j] > -1 and pt[0]+i >= 0 and pt[1]+j >= 0:
                        ret.append((pt[0]+i, pt[1]+j))
                except IndexError as e:
                    pass
    return ret

generate_pts_with_distance_in_row((10,5),2,map,12)

(5, 5)

In [161]:
file1 = open('15.txt', 'r')
lines = file1.readlines()

sensor_beacons = []


for l in lines:
    l = l.replace('\n','').replace(',','').replace(':','')
    row = l.split(' ')
    sensor_beacons.append(((int(row[3][2:]), int(row[2][2:])), (int(row[9][2:]), int(row[8][2:]))))
    # print(row[2][2:],row[3],row[8],row[9])
# sensor_beacons

In [162]:

x_range = (min([pt[0] for pts in sensor_beacons for pt in pts]), max([pt[0] for pts in sensor_beacons for pt in pts]))
y_range = (min([pt[1] for pts in sensor_beacons for pt in pts]), max([pt[1] for pts in sensor_beacons for pt in pts]))

print(x_range, y_range)

(-872454, 4608563) (-1016402, 4322327)


In [163]:
padding = 0

width = x_range[1]-x_range[0]+1+2*padding
height = y_range[1]-y_range[0]+1+2*padding

def transform_coor(pt):
    return (pt[0]-x_range[0]+padding, pt[1]-y_range[0]+padding)

# map = np.zeros((width, height))
map = scipy.sparse.lil_matrix((width, height))

print(width, height)

5481018 5338730


In [164]:
for sensor, beacon in sensor_beacons:
    # print(sensor, beacon)
    map[transform_coor((sensor[0],sensor[1]))] = 5
    map[transform_coor((beacon[0],beacon[1]))] = 1
# pprint(map)

In [8]:
pprint(map)

....S.......................
......................S.....
...............S............
................SB..........
............................
............................
............................
..........S.......S.........
............................
............................
....B.......................
..S.........................
............................
............................
..............S.......S.....
B...........................
...........SB...............
................S..........B
....S.......................
............................
............S......S........
............................
.......................B....


In [165]:
row = 2000000
# row = 10

interval = P.empty()

row_coor = transform_coor((10,0))[0]

for sensor, beacon in sensor_beacons:
    sensor_trans = transform_coor((sensor[0],sensor[1]))
    beacon_trans = transform_coor((beacon[0],beacon[1]))
    distance = abs(sensor[0] - beacon[0]) + abs(sensor[1] - beacon[1])
    # print(sensor, beacon, distance)
    # transform_coor((sensor[0],sensor[1]))
    beg, end = generate_pts_with_distance_in_row((sensor[0],sensor[1]),distance,map, row)
    # print(sensor_trans, beacon_trans, distance)
    # print(P.closed(beg, end))
    # map[row_coor, beg:end+1] = 3
    interval = interval | P.closed(beg, end)
    # print(interval)
    # print(sensor, beacon, distance)

    # for pt in generate_pts_with_distance_in_row(transform_coor((sensor[0],sensor[1])),distance,map, row_coor):
    #     if map[pt] == 0:
    #         map[pt] = 3
    # pprint(map)
    # print()
#count the beacons and sensor loc
cnt = set()
for sensor, beacon in sensor_beacons:
    # print(sensor, beacon)
    if sensor[0] == row:
        cnt = cnt | {sensor}
    if beacon[0] == row:
        cnt = cnt | {beacon}
cnt
# pprint(map)

{(2000000, 1144814)}

In [166]:
eval(str(interval))[1] - eval(str(interval))[0] +1- len(cnt)

5166077

In [141]:
####################################################################

In [183]:
class Beacon():
    def __init__(self, b_x,b_y,s_x,s_y) -> None:
        self.beacon_x = b_x
        self.beacon_y = b_y
        self.sensor_x = s_x
        self.sensor_y = s_y
        self.distance = abs(b_x - s_x) + abs(b_y + s_y)
    
    def cannot_contain_beacon(self, x,y):
        return abs(self.beacon_x - x) + abs(self.beacon_y - y) <= self.distance

file1 = open('15.txt', 'r')
lines = file1.readlines()

beacons = []
things = set()


for l in lines:
    l = l.replace('\n','').replace(',','').replace(':','')
    row = l.split(' ')
    beacons.append(Beacon(int(row[3][2:]), int(row[2][2:]), int(row[9][2:]), int(row[8][2:])))
    things = things | {(int(row[3][2:]), int(row[2][2:])), (int(row[9][2:]), int(row[8][2:]))}
    # sensor_beacons.append(((int(row[3][2:]), int(row[2][2:])), (int(row[9][2:]), int(row[8][2:]))))
    # print(row[3],row[2],row[8],row[9])

# beacons
# things

In [90]:
# row = 10
# ans1 = set()

# for j in range(y_range[0],y_range[1]):
#     if not j % 10000:
#         print(j)
#     for beacon in beacons:
#         if beacon.cannot_contain_beacon(row,j):
#             ans1 = ans1 | {(row,j)}
# ans1 = ans1 - things
# # ans1

# Second part - VERY inefficient. Should rewrite using intervals and finding cells with distance over __

In [171]:
row = 2000000
row = 11

max_coor = 4000000


for row in tqdm(range(0,max_coor+1)):
    interval = P.empty()
    for sensor, beacon in sensor_beacons:
        distance = abs(sensor[0] - beacon[0]) + abs(sensor[1] - beacon[1])
        beg, end = generate_pts_with_distance_in_row((sensor[0],sensor[1]),distance,map, row)
        interval = interval | P.closed(beg, end)
    nums = set()
    for i in list(interval.complement().intersection(P.closed(0,max_coor))):
        interval_py = eval(str(i))
        nums = nums | set(range(interval_py[0]+1,interval_py[1]))
    nums
    if nums:
        print(nums,row)
        for x in nums:
            tunning_freg = 4000000*x+row
            print(f'{tunning_freg=}')
    #count the beacons and sensor loc
    cnt = set()
    for sensor, beacon in sensor_beacons:
        # print(sensor, beacon)
        if sensor[0] == row:
            cnt = cnt | {sensor}
        if beacon[0] == row:
            cnt = cnt | {beacon}
    cnt
# pprint(map)

 68%|██████▊   | 2704932/4000001 [07:06<03:27, 6235.81it/s]

{3267801} 2703981
tunning_freg=13071206703981


 70%|███████   | 2816581/4000001 [07:24<03:06, 6331.55it/s]


KeyboardInterrupt: 